In [5]:
import pandas as pd

In [6]:
data = pd.read_csv('/Users/egorbykov/Desktop/Работа/2025/hackatons/mos/X5_ner_MiLky_way/data/clear_dataset/все_токены_по_отдельности.csv')

In [7]:
data.drop_duplicates(inplace=True)
data

,sample,start,end,type,original_sample
0,aa,0,2,O,aa
1,aala,0,4,O,aala
2,aarcca,0,6,O,aarcca
3,abon,0,4,O,abon
4,abso,0,4,B-BRAND,abso
...,...,...,...,...,...
42248,пиво,3,7,B-TYPE,№1 пиво
42249,№1,0,2,B-BRAND,№1 са
42250,са,3,5,B-TYPE,№1 са
42251,№1,0,2,B-BRAND,№1 саш


In [30]:
test = pd.read_csv('/Users/egorbykov/Desktop/Работа/2025/hackatons/mos/X5_ner_MiLky_way/data/Датасет/submission.csv', sep=';')

In [31]:
test.drop(columns = 'annotation', inplace = True)

In [10]:
sub_data = data[['sample', 'type']]

In [11]:
global_dict = sub_data.set_index('sample').to_dict('dict')['type']

In [12]:
global_dict

{'aa': 'O',
 'aala': 'O',
 'aarcca': 'O',
 'abon': 'O',
 'abso': 'B-BRAND',
 'abtoys': 'B-BRAND',
 'игруш': 'I-TYPE',
 'игрушк': 'I-TYPE',
 'игрушки': 'I-TYPE',
 'acofe': 'O',
 'actimal': 'B-BRAND',
 'actimel': 'B-BRAND',
 'actimeuno': 'B-BRAND',
 'actimino': 'B-BRAND',
 'actimun': 'B-BRAND',
 'actimunno': 'B-BRAND',
 'activ': 'B-BRAND',
 'activa': 'B-BRAND',
 'active': 'B-BRAND',
 'adremaine': 'B-BRAND',
 'adren': 'B-BRAND',
 'adrena': 'B-BRAND',
 'adrenali': 'B-BRAND',
 'adrinalin': 'B-BRAND',
 'adrinaline': 'B-BRAND',
 'afanasiy': 'B-BRAND',
 'пи': 'O',
 'пив': 'B-TYPE',
 'пиво': 'B-TYPE',
 'agama': 'B-BRAND',
 'треск': 'I-TYPE',
 'треска': 'B-TYPE',
 'agata': 'B-BRAND',
 'agnesi': 'B-BRAND',
 'пше': 'B-TYPE',
 'пшен': 'B-TYPE',
 'ahma': 'B-BRAND',
 'ahmad': 'B-BRAND',
 'tea': 'I-BRAND',
 'ailtero': 'B-BRAND',
 'airw': 'B-BRAND',
 'akba': 'B-BRAND',
 'akmad': 'B-BRAND',
 'aktime': 'B-BRAND',
 'aktimel': 'B-BRAND',
 'aktimuno': 'B-BRAND',
 'albeni': 'B-BRAND',
 'albenia': 'B-BRAND',


In [9]:
res = []
for i, row in test.iterrows():
    annotation = []
    for item in row['sample'].split(' '):
        start = row['sample'].index(item)
        end = row['sample'].index(item)+len(item)
        type_token = global_dict.get(item, 'O')
        annotation.append((start, end, type_token))
    res.append(annotation)

In [10]:
test['annotation'] = res

In [11]:
test

,sample,annotation
0,форма для выпечки,"[(0, 5, B-TYPE), (6, 9, O), (10, 17, O)]"
1,фарш свиной,"[(0, 4, B-TYPE), (5, 11, I-TYPE)]"
2,сок ананасовый без сахара,"[(0, 3, B-TYPE), (4, 14, B-TYPE), (15, 18, O),..."
3,еринги,"[(0, 6, O)]"
4,молооко,"[(0, 7, O)]"
...,...,...
4995,milkywa,"[(0, 7, O)]"
4996,очиститель для унитаза,"[(0, 10, B-TYPE), (11, 14, O), (15, 22, O)]"
4997,арбузные,"[(0, 8, O)]"
4998,кашы,"[(0, 4, O)]"


### Левинштейн

In [32]:
import Levenshtein

def find_closest_token(token, token_dict, max_distance=3):
    """
    Находит ближайший токен в словаре по расстоянию Левенштейна
    """
    if not token or not token_dict:
        return 'O'

    # Если токен уже есть в словаре
    if token in token_dict:
        return token_dict[token]

    min_distance = float('inf')
    closest_token = 'O'

    for dict_token in token_dict.keys():
        distance = Levenshtein.distance(token, dict_token)
        if distance < min_distance:
            min_distance = distance
            closest_token = dict_token

    # Если минимальное расстояние превышает порог, возвращаем 'O'
    if min_distance > max_distance:
        return 'O'

    return token_dict[closest_token]

res = []
lens = []
for i, row in test.iterrows():
    annotation = []
    text = row['sample']
    tokens = text.split(' ')

    for item in tokens:
        start = text.find(item)  # Используем find вместо index для безопасности
        if start == -1:
            continue  # Пропускаем если не найдено (маловероятно)

        end = start + len(item)
        type_token = find_closest_token(item, global_dict)
        annotation.append((start, end, type_token))
    lens.append(len(tokens))
    res.append(annotation)

In [33]:
test['annotation'] = res

In [35]:
test

,sample,annotation,lens
0,форма для выпечки,"[(0, 5, B-TYPE), (6, 9, O), (10, 17, O)]",3
1,фарш свиной,"[(0, 4, B-TYPE), (5, 11, I-TYPE)]",2
2,сок ананасовый без сахара,"[(0, 3, B-TYPE), (4, 14, B-TYPE), (15, 18, O),...",4
3,еринги,"[(0, 6, B-BRAND)]",1
4,молооко,"[(0, 7, I-TYPE)]",1
...,...,...,...
4995,milkywa,"[(0, 7, B-BRAND)]",1
4996,очиститель для унитаза,"[(0, 10, B-TYPE), (11, 14, O), (15, 22, O)]",3
4997,арбузные,"[(0, 8, B-TYPE)]",1
4998,кашы,"[(0, 4, I-TYPE)]",1


### Правильные префиксы у Левинштейна

In [37]:
res = []
for i, row in test.iterrows():
    old_type = ''
    annotation = []
    for item in row['annotation']:
        start, end, type_token = item

        if type_token == 'O':
            old_type = type_token
            annotation.append((start, end, type_token))


        elif old_type == '':
            type_token = 'B-'+type_token[2:]
            old_type = type_token
            annotation.append((start, end, type_token))


        elif old_type[2:] == type_token[2:]:
            type_token = 'I-'+type_token[2:]
            old_type = type_token
            annotation.append((start, end, type_token))

        elif old_type[2:] != type_token[2:]:
            type_token = 'B-'+type_token[2:]
            old_type = type_token
            annotation.append((start, end, type_token))
        else:
            print('ELSE')

    res.append(annotation)


In [38]:
test['annotation'] = res

In [49]:
def find_percent_valume(row):
    for item in row:
        if 'PERCENT' in item[2]:
            return True
        if 'VOLUME' in item[2]:
            return True
    return False

In [60]:
for i, row in test[test['annotation'].apply(find_percent_valume)].iterrows():
    print(i, row.values)

906 ['апельсиновый сок 2 литра'
 list([(0, 12, 'B-TYPE'), (13, 16, 'I-TYPE'), (17, 18, 'B-VOLUME'), (19, 24, 'I-VOLUME')])]
1098 ['вода святой источник 5л'
 list([(0, 4, 'B-TYPE'), (5, 11, 'I-TYPE'), (12, 20, 'B-BRAND'), (21, 23, 'B-VOLUME')])]
1162 ['сок сады придонья 1л'
 list([(0, 3, 'B-TYPE'), (4, 8, 'B-BRAND'), (9, 17, 'I-BRAND'), (18, 20, 'B-VOLUME')])]
1173 ['вода большой обьем'
 list([(0, 4, 'B-TYPE'), (5, 12, 'B-BRAND'), (13, 18, 'B-VOLUME')])]
2124 ['сок яблочный 2л'
 list([(0, 3, 'B-TYPE'), (4, 12, 'I-TYPE'), (13, 15, 'B-VOLUME')])]
2403 ['молоко без л' list([(0, 6, 'B-TYPE'), (7, 10, 'O'), (2, 3, 'B-VOLUME')])]
2457 ['сок 2 литра яблочный'
 list([(0, 3, 'B-TYPE'), (4, 5, 'B-VOLUME'), (6, 11, 'I-VOLUME'), (12, 20, 'B-TYPE')])]
2517 ['лютры светилнии' list([(0, 5, 'B-VOLUME'), (6, 15, 'B-TYPE')])]
2550 ['вода 5 литро' list([(0, 4, 'B-TYPE'), (5, 6, 'O'), (7, 12, 'B-VOLUME')])]
2651 ['макароны макфа 16'
 list([(0, 8, 'B-TYPE'), (9, 14, 'B-BRAND'), (15, 17, 'B-VOLUME')])]
2829 

In [52]:
old_test = pd.read_csv('/Users/egorbykov/Desktop/Работа/2025/hackatons/mos/X5_ner_MiLky_way/data/submitions/submission_0.69_levinsh.csv', sep=';')

In [56]:
old_test.iloc[[ 906, 1098, 1162, 1173, 2124, 2403, 2457, 2517, 2550, 2651, 2829, 3393,
       3437, 3449, 3572, 3624, 3870, 3894, 3928, 4649]]

,sample,annotation
906,апельсиновый сок 2 литра,"[(0, 12, 'I-TYPE'), (13, 16, 'B-TYPE'), (17, 1..."
1098,вода святой источник 5л,"[(0, 4, 'B-TYPE'), (5, 11, 'B-TYPE'), (12, 20,..."
1162,сок сады придонья 1л,"[(0, 3, 'B-TYPE'), (4, 8, 'I-BRAND'), (9, 17, ..."
1173,вода большой обьем,"[(0, 4, 'B-TYPE'), (5, 12, 'B-BRAND'), (13, 18..."
2124,сок яблочный 2л,"[(0, 3, 'B-TYPE'), (4, 12, 'B-TYPE'), (13, 15,..."
2403,молоко без л,"[(0, 6, 'I-TYPE'), (7, 10, 'O'), (2, 3, 'I-VOL..."
2457,сок 2 литра яблочный,"[(0, 3, 'B-TYPE'), (4, 5, 'B-VOLUME'), (6, 11,..."
2517,лютры светилнии,"[(0, 5, 'I-VOLUME'), (6, 15, 'I-TYPE')]"
2550,вода 5 литро,"[(0, 4, 'B-TYPE'), (5, 6, 'O'), (7, 12, 'I-VOL..."
2651,макароны макфа 16,"[(0, 8, 'B-TYPE'), (9, 14, 'B-BRAND'), (15, 17..."


In [61]:
updates = {
    906: {'sample': 'апельсиновый сок 2 литра', 'annotation': [(0, 12, 'B-TYPE'), (13, 16, 'I-TYPE'), (17, 18, 'B-VOLUME'), (19, 24, 'I-VOLUME')], 'len_tokens': 4, 'count_spaces': 3},
    1098: {'sample': 'вода святой источник 5л', 'annotation': [(0, 4, 'B-TYPE'), (5, 11, 'B-BRAND'), (12, 20, 'I-BRAND'), (21, 23, 'B-VOLUME')], 'len_tokens': 4, 'count_spaces': 3},
    1162: {'sample': 'сок сады придонья 1л', 'annotation': [(0, 3, 'B-TYPE'), (4, 8, 'B-BRAND'), (9, 17, 'I-BRAND'), (18, 20, 'B-VOLUME')], 'len_tokens': 4, 'count_spaces': 3},
    1173: {'sample': 'вода большой обьем', 'annotation': [(0, 4, 'B-TYPE'), (5, 12, 'B-VOLUME'), (13, 18, 'I-VOLUME')], 'len_tokens': 3, 'count_spaces': 2},
    2124: {'sample': 'сок яблочный 2л', 'annotation': [(0, 3, 'B-TYPE'), (4, 12, 'I-TYPE'), (13, 15, 'B-VOLUME')], 'len_tokens': 3, 'count_spaces': 2},
    2403: {'sample': 'молоко без л', 'annotation': [(0, 6, 'B-TYPE'), (7, 10, 'O'), (2, 3, 'B-VOLUME')], 'len_tokens': 3, 'count_spaces': 2},
    2457: {'sample': 'сок 2 литра яблочный', 'annotation': [(0, 3, 'B-TYPE'), (4, 5, 'B-VOLUME'), (6, 11, 'I-VOLUME'), (12, 20, 'B-TYPE')], 'len_tokens': 4, 'count_spaces': 3},
    2517: {'sample': 'лютры светилнии', 'annotation': [(0, 5, 'B-TYPE'), (6, 15, 'I-TYPE')], 'len_tokens': 2, 'count_spaces': 1},
    2550: {'sample': 'вода 5 литро', 'annotation': [(0, 4, 'B-TYPE'), (5, 6, 'B-VOLUME'), (7, 12, 'I-VOLUME')], 'len_tokens': 3, 'count_spaces': 2},
    2651: {'sample': 'макароны макфа 16', 'annotation': [(0, 8, 'B-TYPE'), (9, 14, 'B-BRAND'), (15, 17, 'B-VOLUME')], 'len_tokens': 3, 'count_spaces': 2},
    2829: {'sample': 'сок 0,2 литра', 'annotation': [(0, 3, 'B-TYPE'), (4, 7, 'B-VOLUME'), (8, 13, 'I-VOLUME')], 'len_tokens': 3, 'count_spaces': 2},
    3393: {'sample': 'сок 2 литра', 'annotation': [(0, 3, 'B-TYPE'), (4, 5, 'B-VOLUME'), (6, 11, 'I-VOLUME')], 'len_tokens': 3, 'count_spaces': 2},
    3437: {'sample': 'хлеб 8 злаков', 'annotation': [(0, 4, 'B-TYPE'), (5, 6, 'I-TYPE'), (7, 13, 'O')], 'len_tokens': 3, 'count_spaces': 2},
    3449: {'sample': 'сахарный песок 5кг', 'annotation': [(0, 8, 'B-TYPE'), (9, 14, 'I-TYPE'), (15, 18, 'B-VOLUME')], 'len_tokens': 3, 'count_spaces': 2},
    3572: {'sample': 'мешки для мусора 120 л', 'annotation': [(0, 5, 'B-TYPE'), (6, 9, 'O'), (10, 16, 'B-TYPE'), (17, 20, 'B-VOLUME'), (7, 8, 'I-VOLUME')], 'len_tokens': 5, 'count_spaces': 4},
    3624: {'sample': 'сок томатный 2 л', 'annotation': [(0, 3, 'B-TYPE'), (4, 12, 'I-TYPE'), (13, 14, 'B-VOLUME'), (15, 16, 'I-VOLUME')], 'len_tokens': 4, 'count_spaces': 3},
    3870: {'sample': 'лимонад 2 литра', 'annotation': [(0, 7, 'B-TYPE'), (8, 9, 'B-VOLUME'), (10, 15, 'I-VOLUME')], 'len_tokens': 3, 'count_spaces': 2},
    3894: {'sample': 'вода 19 л', 'annotation': [(0, 4, 'B-TYPE'), (5, 7, 'B-VOLUME'), (8, 9, 'I-VOLUME')], 'len_tokens': 3, 'count_spaces': 2},
    3928: {'sample': 'вода питьевая большой обьем', 'annotation': [(0, 4, 'B-TYPE'), (5, 13, 'I-TYPE'), (14, 21, 'B-VOLUME'), (22, 27, 'I-VOLUME')], 'len_tokens': 4, 'count_spaces': 3},
    4649: {'sample': 'молоко 1,5 %', 'annotation': [(0, 6, 'B-TYPE'), (7, 10, 'B-PERCENT'), (11, 12, 'I-PERCENT')], 'len_tokens': 3, 'count_spaces': 2}
}
# Обновляем данные
for idx, values in updates.items():
    for col, value in values.items():
        if not col in  ['sample', 'annotation']: continue
        test.at[idx, col] = value

### Сохранение

In [62]:
test.to_csv('/Users/egorbykov/Desktop/Работа/2025/hackatons/mos/X5_ner_MiLky_way/data/submitions/submission.csv', index=False, sep=';', columns = ['sample', 'annotation'])